# Layer Norm with Bitmask Generation

This notebook shows how to compute forward Layer Norm (training) + clamped ReLU (with bitmask generation), then compute the backward equivalent (DReLU + DLN) using the bitmask. 

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/NVIDIA/cudnn-frontend/blob/main/samples/python/25_layernorm_forward_training_and_backward_with_relu_bitmask.ipynb)

## Prerequisites and Setup
This notebook requires an NVIDIA GPU. If `nvidia-smi` fails, go to Runtime -> Change runtime type -> Hardware accelerator and confirm a GPU is selected.

In [ ]:
#get_ipython().system('nvidia-smi')

If running on Colab, you will need to install the cudnn python interface.

In [ ]:
# get_ipython().system('pip install nvidia-cudnn-cu12')
# get_ipython().system('pip install nvidia-cudnn-frontend')
# get_ipython().system('pip3 install --pre torch --index-url https://download.pytorch.org/whl/nightly/cu128')

## Overview

In the following, we will apply layer norm to a tensor of the following shape:

- Batch Size: 4
- Sequence Size: 1024
- Embedding Dimension: 768

Let's define these dimensions as constants:

In [ ]:
import cudnn
import torch

torch.manual_seed(1)

handle = cudnn.create_handle()
print("Running with cudnn backend version:", cudnn.backend_version())

assert torch.cuda.is_available()
assert (
    cudnn.backend_version() >= 91300
), "LayerNorm with relu bitmask generation is only supported cuDNN version 9.13.0 or above"

batch, seq_size, embedding_dim = 4, 1024, 768
dtype = torch.float32
# Epsilon is a small number to prevent division by 0.
epsilon_value = 1e-3
# Set clamped ReLU limits
lower_clip_val = 0
upper_clip_val = 6

Additionally, we need to define a helper function to unpack a uint8 bitmask tensor into a boolean tensor. This is to be used when we want to examine the bitmask tensor against the boolean result as computed by PyTorch:

In [ ]:
def unpack_cudnn_bitmask(bitmask_tensor, N, C, H=1, W=1):
    """
    Helper function to unpack a bitmask tensor of shape [N, C//8, H, W] and dtype=torch.uint8
    (stored as packed bits) into a boolean tensor of shape [N, C, H, W] for assert testing.
    """
    packed = bitmask_tensor.view(N, C // 8, H * W)
    unpacked = torch.zeros((N, C, H * W), dtype=torch.bool, device=packed.device)

    # populate each bit of the unpacked tensor
    for bit in range(8):
        bit_values = (packed >> bit) & 1
        unpacked[:, bit::8, :] = bit_values

    unpacked = unpacked.view(N, C, H, W)
    return unpacked

## Using Wrapper

#### LayerNorm Relu Bitmask Training Forward Pass

First, we define the input tensors

In [ ]:
# random tensors as input
x_gpu = torch.randn(
    batch * seq_size,
    embedding_dim,
    1,
    1,
    device="cuda",
    dtype=dtype,
    requires_grad=True,
).to(memory_format=torch.channels_last)
scale_gpu = torch.randn(
    1, embedding_dim, 1, 1, device="cuda", dtype=dtype, requires_grad=True
).to(memory_format=torch.channels_last)
bias_gpu = torch.randn(
    1, embedding_dim, 1, 1, device="cuda", dtype=dtype, requires_grad=True
).to(memory_format=torch.channels_last)

# Epsilon, lower clip, and upper clip must be a scalar value on the cpu.
epsilon_cpu = torch.full(
    (1, 1, 1, 1), epsilon_value, dtype=torch.float32, requires_grad=False, device="cpu"
)
lower_clip_cpu = torch.full(
    (1, 1, 1, 1), lower_clip_val, dtype=torch.float32, requires_grad=False, device="cpu"
)
upper_clip_cpu = torch.full(
    (1, 1, 1, 1), upper_clip_val, dtype=torch.float32, requires_grad=False, device="cpu"
)

Next, create the graph for the forward pass.

In [ ]:
with cudnn.Graph(
    io_data_type=cudnn.data_type.FLOAT,
    intermediate_data_type=cudnn.data_type.FLOAT,
    compute_data_type=cudnn.data_type.FLOAT,
) as fwd_graph:
    # layernorm forward pass
    norm_out, mean, inv_var = fwd_graph.layernorm(
        name="LN",
        norm_forward_phase=cudnn.norm_forward_phase.TRAINING,
        input=x_gpu,
        scale=scale_gpu,
        bias=bias_gpu,
        epsilon=epsilon_cpu,
    )
    # relu on the layernorm output
    out = fwd_graph.relu(
        name="ReLU",
        input=norm_out,
        lower_clip=lower_clip_val,
        upper_clip=upper_clip_val,
    )
    # generate bitmask with the ReLU output
    lower_clip_mask = fwd_graph.cmp_gt(
        name="lower_mask",
        input=out,
        comparison=lower_clip_cpu,
    )
    lower_clip_mask.set_name("lower_clip").set_data_type(cudnn.data_type.BOOLEAN)
    upper_clip_mask = fwd_graph.cmp_lt(
        name="upper_mask",
        input=out,
        comparison=upper_clip_cpu,
    )
    upper_clip_mask.set_name("upper_clip").set_data_type(cudnn.data_type.BOOLEAN)
    bitmask = fwd_graph.logical_and(
        name="and_bitmask",
        a=lower_clip_mask,
        b=upper_clip_mask,
    )
    # mark the output tensors
    out.set_name("output").set_output(True)
    mean.set_name("mean").set_output(True).set_data_type(cudnn.data_type.FLOAT)
    inv_var.set_name("inv_var").set_output(True).set_data_type(cudnn.data_type.FLOAT)
    bitmask.set_name("relu_bitmask").set_output(True).set_data_type(
        cudnn.data_type.BOOLEAN
    )

Then, execute the graph and compare the output to the reference output from PyTorch:

In [ ]:
# allocated output tensors
out_gpu = torch.empty_like(x_gpu)
mean_gpu = torch.empty(batch * seq_size, 1, 1, 1, dtype=torch.float32, device="cuda")
inv_var_gpu = torch.empty(batch * seq_size, 1, 1, 1, dtype=torch.float32, device="cuda")
# CuDNN stores boolean bitmask values as bit-packed int8_t.
mask_gpu = torch.empty(
    ((batch * seq_size), embedding_dim // 8, 1, 1), dtype=torch.uint8, device="cuda"
)

# execute the graph
output = fwd_graph(
    {
        # input tensors
        "LN::input": x_gpu,
        "LN::scale": scale_gpu,
        "LN::bias": bias_gpu,
        "LN::epsilon": epsilon_cpu,
        "lower_mask::comparison": lower_clip_cpu,
        "upper_mask::comparison": upper_clip_cpu,
        # output tensors
        "output": out_gpu,
        "mean": mean_gpu,
        "inv_var": inv_var_gpu,
        "relu_bitmask": mask_gpu,
    },
    handle=handle,
)

# PyTorch reference forward operation
normalized_x = torch.nn.functional.layer_norm(
    x_gpu,
    [embedding_dim, 1, 1],
    weight=scale_gpu.squeeze(0),
    bias=bias_gpu.squeeze(0),
    eps=epsilon_value,
)
out_ref = torch.clamp(normalized_x, min=lower_clip_val, max=upper_clip_val)
mask_ref = (lower_clip_val < out_ref) & (out_ref < upper_clip_val)
mean_ref = x_gpu.to(torch.float32).mean(dim=(1, 2, 3), keepdim=True)
inv_var_ref = torch.rsqrt(
    torch.var(x_gpu.to(torch.float32), dim=(1, 2, 3), keepdim=True) + epsilon_value
)

# compare to reference output
torch.testing.assert_close(out_gpu, out_ref, rtol=5e-3, atol=5e-3)
torch.testing.assert_close(inv_var_gpu, inv_var_ref, rtol=5e-3, atol=5e-3)
torch.testing.assert_close(mean_gpu, mean_ref, rtol=5e-3, atol=5e-3)

# Unpack the bitmask tensor and compare to reference output
unpacked_mask = unpack_cudnn_bitmask(mask_gpu, batch * seq_size, embedding_dim, 1, 1)
torch.testing.assert_close(unpacked_mask, mask_ref, atol=1e-3, rtol=1e-3)

#### LayerNorm Relu Bitmask Backward Pass

In [ ]:
# Reference backward operation using PyTorch
target = torch.randn_like(out_ref)
criterion = torch.nn.MSELoss()
loss = criterion(out_ref, target)

out_ref.retain_grad()
x_gpu.retain_grad()
scale_gpu.retain_grad()
bias_gpu.retain_grad()

loss.backward()

In [ ]:
# Backward pass
with cudnn.Graph(
    intermediate_data_type=cudnn.data_type.FLOAT,
    compute_data_type=cudnn.data_type.FLOAT,
    inputs=[
        "drelu_bitmask_mul::a",
        "drelu_bitmask_mul::b",
        "DLN::input",
        "DLN::scale",
        "DLN::mean",
        "DLN::inv_variance",
    ],
    outputs=["dX", "dScale", "dBias"],
) as bwd_graph:
    # pointwise mul operation for dRelu using the bitmask
    drelu_dY = bwd_graph.mul(
        name="drelu_bitmask_mul",
        a=out_ref.grad,
        b=mask_ref,
    )
    # the layernorm backward operation
    d_x, d_scale, d_bias = bwd_graph.layernorm_backward(
        name="DLN",
        grad=drelu_dY,
        input=x_gpu,
        scale=scale_gpu,
        mean=mean_gpu,
        inv_variance=inv_var_gpu,
    )
    # mark the output tensors
    d_x.set_output(True).set_name("dX").set_data_type(dtype)
    d_scale.set_output(True).set_name("dScale").set_data_type(dtype)
    d_bias.set_output(True).set_name("dBias").set_data_type(dtype)

# Execute the backward graph
d_x_gpu, d_scale_gpu, d_bias_gpu = bwd_graph(
    out_ref.grad,
    mask_gpu.detach(),
    x_gpu.detach(),
    scale_gpu.detach(),
    mean_gpu.detach(),
    inv_var_gpu.detach(),
    handle=handle,
)

# compare to reference output
torch.testing.assert_close(x_gpu.grad, d_x_gpu, atol=2e-4, rtol=2e-4)
torch.testing.assert_close(scale_gpu.grad, d_scale_gpu, atol=2e-4, rtol=2e-4)
torch.testing.assert_close(bias_gpu.grad, d_bias_gpu, atol=2e-4, rtol=2e-4)

## Using Python Binding APIs

#### LayerNorm ReLU Bitmask Training Forward Pass

Create input tensor GPU buffers. We use PyTorch to allocate GPU tensors so we can reuse them easily when we calculate reference outputs.

In [ ]:
# Allocate input tensor memory, initialize them to random numbers
x_gpu = torch.randn(
    batch * seq_size,
    embedding_dim,
    1,
    1,
    device="cuda",
    dtype=dtype,
    requires_grad=True,
).to(memory_format=torch.channels_last)
scale_gpu = torch.randn(
    1, embedding_dim, 1, 1, device="cuda", dtype=dtype, requires_grad=True
).to(memory_format=torch.channels_last)
bias_gpu = torch.randn(
    1, embedding_dim, 1, 1, device="cuda", dtype=dtype, requires_grad=True
).to(memory_format=torch.channels_last)

# Epsilon, lower clip, and upper clip must be a scalar value on the cpu.
epsilon_cpu = torch.full(
    (1, 1, 1, 1), epsilon_value, dtype=torch.float32, requires_grad=False, device="cpu"
)
lower_clip_cpu = torch.full(
    (1, 1, 1, 1), lower_clip_val, dtype=torch.float32, requires_grad=False, device="cpu"
)
upper_clip_cpu = torch.full(
    (1, 1, 1, 1), upper_clip_val, dtype=torch.float32, requires_grad=False, device="cpu"
)

Then we create the graph for the forward pass.

In [ ]:
# Create the cuDNN graph.
graph = cudnn.pygraph(
    handle=handle,
    io_data_type=cudnn.data_type.FLOAT,
    intermediate_data_type=cudnn.data_type.FLOAT,
    compute_data_type=cudnn.data_type.FLOAT,
)

# Create tensor handles with the graph API, assign UIDs.
x = graph.tensor_like(x_gpu.detach()).set_name("X")
scale = graph.tensor_like(scale_gpu.detach()).set_name("scale")
bias = graph.tensor_like(bias_gpu.detach()).set_name("bias")
epsilon = graph.tensor_like(epsilon_cpu).set_name("epsilon")
lower_clip = graph.tensor_like(lower_clip_cpu).set_name("lower_clip")
upper_clip = graph.tensor_like(upper_clip_cpu).set_name("upper_clip")

# Add a layernorm operation
norm_out, mean, inv_var = graph.layernorm(
    name="layernorm",
    norm_forward_phase=cudnn.norm_forward_phase.TRAINING,
    input=x,
    scale=scale,
    bias=bias,
    epsilon=epsilon,
)

# Add a relu operation
out = graph.relu(
    name="relu", input=norm_out, lower_clip=lower_clip_val, upper_clip=upper_clip_val
)

# Add logical operations for generating bitmask
lower_clip_mask = graph.cmp_gt(
    name="cmp_gt_lower_clip", input=out, comparison=lower_clip
)
lower_clip_mask.set_name("lower_clip").set_data_type(cudnn.data_type.BOOLEAN)
upper_clip_mask = graph.cmp_lt(
    name="cmp_lt_upper_clip", input=out, comparison=upper_clip
)
upper_clip_mask.set_name("upper_clip").set_data_type(cudnn.data_type.BOOLEAN)
bitmask = graph.logical_and(name="and_bitmask", a=lower_clip_mask, b=upper_clip_mask)
bitmask.set_name("upper_clip").set_data_type(cudnn.data_type.BOOLEAN)

# Enable all outputs, by default outputs are disabled
out.set_name("output").set_output(True)
mean.set_name("mean").set_output(True).set_data_type(cudnn.data_type.FLOAT)
inv_var.set_name("inv_var").set_output(True).set_data_type(cudnn.data_type.FLOAT)
bitmask.set_name("relu_bitmask").set_output(True)

# print(graph)

# Build the graph
graph.build([cudnn.heur_mode.A, cudnn.heur_mode.FALLBACK])

Here we assign UIDs for tensors. UIDs are a unique identifier that will allow us to provide a mapping from tensors created from cuDNN graph api calls, such as `graph.tensor_like()`, to the underlying device memory that will be used to store these tensors. Virtual tensors don't require explicit memory allocated for them, but non-vritual tensors like inputs or outputs will need to have UIDs assigned to them. 

Alternatively, one can use handles directly in the mapping, however using UIDs can be more convinient for caching of cuDNN graphs.

For each of our inputs {X, Scale, Bias, Epsilon} and our outputs {Out, Mean, Inverse Variance}, we allocate a UID. 

After validating and building a cuDNN graph,  we can now execute it. To do this, we have to provide input and output buffers. We do this by using the previously allocated UIDs to associate between tensor handles generated from the graph API, and their underlying memory. 

The desired input values need to be stored in these buffers before the `graph.execute` call. Because we have done a reference computation, we can simply reuse the buffers we have allocated via PyTorch.

Note that the EPISLON UID expects a cpu buffer, 

In [ ]:
# Allocate output tensor memory.
out_gpu = torch.empty_like(x_gpu)
mean_gpu = torch.empty(batch * seq_size, dtype=torch.float32, device="cuda")
inv_var_gpu = torch.empty(batch * seq_size, dtype=torch.float32, device="cuda")

# CuDNN stores boolean bitmask values as bit-packed int8_t.
mask_gpu = torch.empty(
    ((batch * seq_size), embedding_dim // 8, 1, 1), dtype=torch.uint8, device="cuda"
)

# mapping of handles -> memory
variant_pack = {
    x: x_gpu,
    scale: scale_gpu,
    bias: bias_gpu,
    epsilon: epsilon_cpu,
    out: out_gpu,
    mean: mean_gpu,
    inv_var: inv_var_gpu,
    lower_clip: lower_clip_cpu,
    upper_clip: upper_clip_cpu,
    bitmask: mask_gpu,
}
workspace = torch.empty(graph.get_workspace_size(), device="cuda", dtype=torch.uint8)
graph.execute(variant_pack, workspace)
torch.cuda.synchronize()

Compute reference ouputs.

In [ ]:
# Reference forward operation using PyTorch
normalized_x = torch.nn.functional.layer_norm(
    x_gpu,
    [embedding_dim, 1, 1],
    weight=scale_gpu.squeeze(0),
    bias=bias_gpu.squeeze(0),
    eps=epsilon_value,
)
out_ref = torch.clamp(normalized_x, min=lower_clip_val, max=upper_clip_val)
mask_ref = (lower_clip_val < out_ref) & (out_ref < upper_clip_val)
mean_ref = x_gpu.to(torch.float32).mean(dim=(1, 2, 3))
inv_var_ref = torch.rsqrt(
    torch.var(x_gpu.to(torch.float32), dim=(1, 2, 3)) + epsilon_value
)

Test cuDNN's output against PyTorch's and check correctness

In [ ]:
# compare to reference output
torch.testing.assert_close(out_gpu, out_ref, rtol=5e-3, atol=5e-3)
torch.testing.assert_close(inv_var_gpu, inv_var_ref, rtol=5e-3, atol=5e-3)
torch.testing.assert_close(mean_gpu, mean_ref, rtol=5e-3, atol=5e-3)

# Unpack the bitmask tensor and compare to reference output
unpacked_mask = unpack_cudnn_bitmask(mask_gpu, batch * seq_size, embedding_dim, 1, 1)
torch.testing.assert_close(unpacked_mask, mask_ref, atol=1e-3, rtol=1e-3)

#### LayerNorm Relu Bitmask Backward Pass


Compute references values for backward graph

In [ ]:
# Reference backward operation using PyTorch
target = torch.randn_like(out_ref)
criterion = torch.nn.MSELoss()
loss = criterion(out_ref, target)

out_ref.retain_grad()
x_gpu.retain_grad()
scale_gpu.retain_grad()
bias_gpu.retain_grad()

loss.backward()

Create cuDNN graph and tensors

In [ ]:
bwd_graph = cudnn.pygraph(
    handle=handle,
    intermediate_data_type=cudnn.data_type.FLOAT,
    compute_data_type=cudnn.data_type.FLOAT,
)

# Create tensors associated with the backwards graph. DO NOT reuse tensor handles from the forward graph.
d_out = bwd_graph.tensor(
    name="d_out", dim=x_gpu.size(), stride=x_gpu.stride(), data_type=x_gpu.dtype
)

x_bwd = bwd_graph.tensor_like(x, name="x")
scale_bwd = bwd_graph.tensor_like(scale, name="scale")
mean_bwd = bwd_graph.tensor_like(mean, name="mean")
inv_var_bwd = bwd_graph.tensor_like(inv_var, name="inv_var")
bitmask_bwd = bwd_graph.tensor(
    name="bitmask",
    dim=(batch * seq_size, embedding_dim, 1, 1),
    stride=(embedding_dim, 1, 1, 1),
    data_type=cudnn.data_type.BOOLEAN,
)

# a pointwise mul operation for dRelu using the bitmask
drelu_dY = bwd_graph.mul(name="drelu_bitmask_mul", a=d_out, b=bitmask_bwd)
drelu_dY.set_name("dRelu(dY)")
print("drelu_x_bwd:", drelu_dY.get_dim())

# the layernorm backward operation
d_x, d_scale, d_bias = bwd_graph.layernorm_backward(
    name="DLN",
    grad=drelu_dY,
    input=x_bwd,
    scale=scale_bwd,
    mean=mean_bwd,
    inv_variance=inv_var_bwd,
)

# Enable outputs.
d_x.set_output(True).set_data_type(x_gpu.dtype)
d_scale.set_output(True).set_data_type(x_gpu.dtype)
d_bias.set_output(True).set_data_type(x_gpu.dtype)

# print(bwd_graph)

# Build the bwd_graph
bwd_graph.build([cudnn.heur_mode.A, cudnn.heur_mode.FALLBACK])

Execute the graph 

In [ ]:
# Create output buffers for gradients
d_x_gpu = torch.empty_like(x_gpu)
d_scale_gpu = torch.empty_like(scale_gpu)
d_bias_gpu = torch.empty_like(bias_gpu)

workspace = torch.empty(
    bwd_graph.get_workspace_size(), device="cuda", dtype=torch.uint8
)

# For the inputs of the backwards graph (x_bwd, d_out, scale_bwd, mean_bwd, inv_var_bwd), we use the outputs of the forwards graph. For d_out we use pytorches autograd .grad functionality.
variant_pack = {
    x_bwd: x_gpu.detach(),
    scale_bwd: scale_gpu.detach(),
    d_out: out_ref.grad,
    mean_bwd: mean_gpu.detach(),
    inv_var_bwd: inv_var_gpu.detach(),
    d_x: d_x_gpu,
    d_scale: d_scale_gpu,
    d_bias: d_bias_gpu,
    bitmask_bwd: mask_gpu.detach(),
}
bwd_graph.execute(variant_pack, workspace)
torch.cuda.synchronize()

Test cuDNN's output against PyTorch's and check correctness

In [ ]:
# compare to reference output
torch.testing.assert_close(x_gpu.grad, d_x_gpu, atol=2e-4, rtol=2e-4)
torch.testing.assert_close(scale_gpu.grad, d_scale_gpu, atol=2e-4, rtol=2e-4)
torch.testing.assert_close(bias_gpu.grad, d_bias_gpu, atol=2e-4, rtol=2e-4)

Perform Cleanup

In [ ]:
cudnn.destroy_handle(handle)